---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [3]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)
df = df.drop('Totals')
df["Diff"]= pd.Series(df["Gold"]-df["Gold.1"],index=df.index)
df["Bakchodi"]=pd.Series(df["Diff"]/df["Gold.2"],index=df.index)
df.reset_index()
df.head()


,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID,Diff,Bakchodi
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG,0,NaN
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG,5,1.0
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG,18,1.0
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM,1,1.0
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ,3,1.0


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [4]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Diff                0
Bakchodi          NaN
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [5]:
def answer_one():
    return df[df["Gold"] == max(df["Gold"])].index.map(str)[0]
answer_one()



'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [7]:
def answer_two():
    return df[df["Gold"]-df["Gold.1"]==max(df["Gold"]-df["Gold.1"])].index.map(str)[0]
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [6]:
def answer_three():
    return df[((df["Gold"]>0) & (df["Gold.1"]>0) ) & (df["Bakchodi"] == 0.96153846153846156) ].index.map(str)[0]
    

answer_three()


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [8]:
def answer_four():
    points=pd.Series(df["Gold.2"]*3+df["Silver.2"]*2+df["Bronze.2"]*1)
    return points
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [22]:
census_df = pd.read_csv('census.csv')
census_df.head()
census_df['MAXPOL']=census_df[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)
census_df['MINPOL']=census_df[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
census_df['Bakchod']=census_df['MAXPOL']-census_df['MINPOL']
c_df=census_df[['SUMLEV','CTYNAME','Bakchod']]
county_df=c_df[c_df['SUMLEV']==50]
county_df

,SUMLEV,CTYNAME,Bakchod
1,50,Autauga County,687
2,50,Baldwin County,20516
3,50,Barbour County,852
4,50,Bibb County,349
5,50,Blount County,403
6,50,Bullock County,281
7,50,Butler County,790
8,50,Calhoun County,2817
9,50,Chambers County,160
10,50,Cherokee County,225


In [10]:
def answer_five():
    return census_df.groupby('STNAME').COUNTY.nunique()[census_df.groupby('STNAME').COUNTY.nunique()==census_df.groupby('STNAME').COUNTY.nunique().max()].index.map(str)[0]
answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [11]:
def answer_six():
    return census_df.groupby(['STNAME','CTYNAME']).CENSUS2010POP.nunique()
answer_six()

STNAME     CTYNAME           
Alabama    Alabama               1
           Autauga County        1
           Baldwin County        1
           Barbour County        1
           Bibb County           1
           Blount County         1
           Bullock County        1
           Butler County         1
           Calhoun County        1
           Chambers County       1
           Cherokee County       1
           Chilton County        1
           Choctaw County        1
           Clarke County         1
           Clay County           1
           Cleburne County       1
           Coffee County         1
           Colbert County        1
           Conecuh County        1
           Coosa County          1
           Covington County      1
           Crenshaw County       1
           Cullman County        1
           Dale County           1
           Dallas County         1
           DeKalb County         1
           Elmore County         1
           Escambia Count

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [24]:
def answer_seven():
    return county_df[ (county_df['Bakchod']==max(county_df['Bakchod']))]['CTYNAME'].to_string()[-13:]
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [13]:
def answer_eight():
    return census_df[((census_df['REGION']==1) | (census_df['REGION'] == 2)) & (census_df['POPESTIMATE2015']>census_df['POPESTIMATE2014']) & (census_df['CTYNAME'].str.contains('Washington') ) ][["STNAME",'CTYNAME']]
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
